In [9]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 2.7.15+ (default, Nov 27 2018, 23:36:35) 
[GCC 7.3.0]
NLTK: 3.2.5
Scikit-learn: 0.20.3
Pandas: 0.24.2
Numpy: 1.14.1


In [4]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_tratado.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null bool
1    793 non-null object
dtypes: bool(1), object(1)
memory usage: 7.0+ KB
None
       0                                                  1
0  False  inexecuca contrat sanca afast reexam prov clau...
1  False  acident aer decretol n dol eventual respons at...
2  False  intimaca art cpc irregular nao comprov recurs ...
3  False  honorari execuca legitim part superior tribuna...
4  False  servidor public artig lei n artig lei n revoga...


In [5]:
# verificação das classes dos acórdãos
classes = df[0]
print(classes.value_counts())

False    525
True     268
Name: 0, dtype: int64


In [6]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes em binário
encoder = LabelEncoder()
binary = encoder.fit_transform(classes)

print(binary[:10])

[0 0 0 0 0 0 1 1 0 0]


In [7]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    inexecuca contrat sanca afast reexam prov clau...
1    acident aer decretol n dol eventual respons at...
2    intimaca art cpc irregular nao comprov recurs ...
3    honorari execuca legitim part superior tribuna...
4    servidor public artig lei n artig lei n revoga...
5    iss arrend mercantil incidenc mat constitucion...
6    processual penal art codig process penal conhe...
7    recurs especial art incis art ambos codig pena...
8    praz prescricional lei complement n artig lei ...
9    licitaca aca anulator indefer tutel antecip ag...
Name: 1, dtype: object


In [10]:
from nltk.tokenize import word_tokenize

# criação do BOW (bag of words)
all_words = []

for ementa in ementas:
    words = word_tokenize(ementa)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [12]:
# Exibição do número total de palavras e as 15 mais comuns
print('Número de palavras: {}'.format(len(all_words)))
print('Palavras mais comuns: {}'.format(all_words.most_common(15)))

Número de palavras: 5084
Palavras mais comuns: [(u'nao', 1735), (u'recurs', 950), (u'agrav', 851), (u'art', 812), (u'n', 790), (u'especial', 782), (u'lei', 779), (u'embarg', 777), (u'sumul', 608), (u'regimental', 512), (u'tribunal', 481), (u'stj', 454), (u'ser', 435), (u'acorda', 384), (u'cort', 367)]


In [13]:
# Uso das 1500 palavras mais comuns como características
word_features = list(all_words.keys())[:1500]

In [14]:
# Função determina quais das 1500 palavras características estão contidas nas ementas

def find_features(ementa):
    words = word_tokenize(ementa)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Por exemplo
features = find_features(ementas[0])
for key, value in features.items():
    if value == True:
        print(key)

aut
med
total
conjunt
inexecuca
contratu
cont
estadual
agrav
deslind
clausul
extrem


In [15]:
# Aplicando à todas as ementas
ementas_zip = list(zip(ementas, binary))
zipped_list_1 = ementas_zip[:]
zipped_list_2 = list(ementas_zip) 

# embaralhando lista
seed = 1
np.random.seed = seed
np.random.shuffle(zipped_list_2)

# chamando a função find_features para cada ementa
featuresets = [(find_features(text), label) for (text, label) in ementas_zip]

In [16]:
# Separação dos conjuntos de característas em treinamento e teste usando sklean
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [17]:
print(len(training))
print(len(testing))

594
199


In [22]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    nltk_model = SklearnClassifier(classifier)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 62.8140703518
Decision Tree Accuracy: 77.8894472362
Random Forest Accuracy: 78.8944723618
Logistic Regression Accuracy: 78.8944723618
SGD Classifier Accuracy: 73.3668341709
Naive Bayes Accuracy: 74.3718592965
SVM Linear Accuracy: 73.8693467337


In [24]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 73.8693467337


In [25]:
# criação de classe de label de predição para conjunto de teste

txt_features_2, labels_2 = list(zip(*testing))
txt_features_1, labels_1 = txt_features_2[:], labels_2[:]
txt_features, labels = list(txt_features_2), list(labels_2)

prediction = nltk_ensemble.classify_many(txt_features)

In [29]:
# relatório de classificação

print(classification_report(labels, prediction))
print(accuracy)

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       136
           1       0.68      0.62      0.65        63

   micro avg       0.79      0.79      0.79       199
   macro avg       0.76      0.74      0.75       199
weighted avg       0.78      0.79      0.79       199

73.86934673366834
